In [1]:
%matplotlib inline
from __future__ import division

import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import numpy as np
from operator import truediv
from nxpd import draw
from collections import Counter
import itertools
import random

In [10]:
BREAKOUT_REQ_RATIO = 2
BREAKOUT_REQ_FRACTION = 0.2
G = nx.DiGraph()

In [11]:
def insert_node(name, email, learn, teach):
    G.add_node(name, email=email, learn=learn, teach=teach)
    
    for node in G.nodes(data=True):
        learn_other = node[1]['learn']
        teach_other = node[1]['teach']
        
        common = [l for l in learn if l in teach_other]
        w = len(common)
        G.add_edge(name, node[0], weight=w, common=common)
        
        common = [l for l in teach if l in learn_other]
        w = len(common)
        G.add_edge(node[0], name, weight=w, common=common)

In [12]:
# Retrieve data from Google Sheet and parse using pandas dataframe
data = pd.read_csv("https://raw.githubusercontent.com/benelson/collaboratr/master/collaboratr.csv")
data = data.replace(np.nan,' ', regex=True)
users = data['1. What is your name?']
emails = data['2. What is your email?']
teach = data['3. What can you help others with?']
learn = data['4. What do you want to collaborate on or learn about?']

## Decision Tree

In [17]:
def find_breakouts(teach, learn):
    """
    Find subjects which deserve breakouts depending 
    on student to teach ratios.
    """
    big_list_teach = []
    for i in range(len(teach)):
        big_list_teach.append(teach[i].split(';'))

    big_list_learn = []
    for i in range(len(learn)):
        big_list_learn.append(learn[i].split(';'))
        
    teach_dict = dict(Counter(list((itertools.chain.from_iterable(big_list_teach)))))
    learn_dict = dict(Counter(list((itertools.chain.from_iterable(big_list_learn)))))
    
    common_names = [ n for n in set(teach_dict).intersection(set(learn_dict)) ]

    teach_dict = {k:teach_dict[k] for k in common_names if k in teach_dict} # Python3
    learn_dict = {k:learn_dict[k] for k in common_names if k in learn_dict} # Python3
    
    N_teach = list(teach_dict.values())
    N_learn = list(learn_dict.values())
    
    ratios =  list(map(truediv,N_learn,N_teach))

    N = len(users)
    subjects = list(teach_dict.keys())
    isBreakout = (np.array(ratios) > BREAKOUT_REQ_RATIO) & (np.array(N_learn) > BREAKOUT_REQ_FRACTION*N)
    breakouts = np.array(subjects)[list(np.where(isBreakout)[0])]
    
    return list(breakouts)

In [23]:
breakouts = find_breakouts(teach,learn)

## Graphs

In [24]:
# Insert users in graphs
for u,e,l,t in zip(users, emails, learn, teach):
    insert_node(u, email=e, learn=l.split(';'), teach=t.split(';'))

In [25]:
participants = G.nodes(data=True)

In [26]:
def assign_users():
    """
    This uses LOTS of nested loops. Work to be done here.
    """
    assign = {}
    i=0
    for p in participants:
        assign_p = {}
        for l in p[1]['learn']:
            if l in breakouts:
                assign_p[l] = 'breakout'
            else:
                edges = G.edges(p[0], data=True)
                common = []
                
                for e in edges:
                    if (l in e[2]['common']) and (l!=''):
                        common.append(e[1])
                    else: 
                        pass
                
                if common != []:
                    assign_p[l] = random.choice(common)   
                
        assign[p[0]] = assign_p
        i+=1
        
    return assign

In [27]:
assign = assign_users()

In [28]:
G_new = nx.DiGraph(bgcolor='white')

In [29]:
G_new.add_nodes_from(participants, style='filled', fillcolor='black', 
                     fontcolor='white', fontname='sans-serif', fontsize=24)
G_new.add_node('breakout', shape='rectangle', width=4, height=1, style='filled', fillcolor='black',
              fontcolor='white', fontname='sans-serif', fontsize=30)

In [30]:
for p in G.nodes():
    mappings = assign[p]
    for m in mappings.keys():
        G_new.add_edge(p, mappings[m], label=m, style='dotted', fontcolor='red', fontsize=20,
                      fontname='sans-serif')

In [31]:
draw(G_new, filename='Resources/0.svg')

'Resources/0.svg'

# Customized Graphs

In [202]:
participants = G_new.nodes()
connections = G_new.edges(data=True)

In [203]:
index=1
for p in participants:
    con = [c for c in connections if c[0] == p]
    links = [c[1] for c in con]
    links.append(p)

    H = nx.DiGraph()
    H.add_nodes_from(links, style='filled', fillcolor='black', 
                         fontcolor='white', fontname='sans-serif', fontsize=24)

    for c in con:
        H.add_edge(c[0], c[1], label=c[2]['label'], style='dotted', fontcolor='red', fontsize=12,
                          fontname='sans-serif')

    draw(H, filename='Resources/'+str(index)+'.svg')
    index+=1

In [204]:
obj = {}
obj['index'] = index
obj['participants'] = [p.split(' ')[0] for p in participants]

# Creating a Web Page

In [205]:
from jinja2 import Template

In [226]:
# Define jinja2 template
TEMPLATE1 = Template("""<html><head><link rel="stylesheet" href="http://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css">

<!-- jQuery library -->
<script src="https://ajax.googleapis.com/ajax/libs/jquery/1.12.4/jquery.min.js"></script>

<!-- Latest compiled JavaScript -->
<script src="http://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/js/bootstrap.min.js"></script></head>
<body>
<nav class="navbar navbar-default">
  <div class="container-fluid">
    <div class="navbar-header">
      <a class="navbar-brand" href="#">Collaboratr</a>
    </div>
    <ul class="nav navbar-nav">
      <li class="active"><a href="#">Home</a></li>
      <li><a href="#0">Overall</a></li>
      {% for p in obj.participants%}
          <li><a href="#{{loop.index}}">{{p}}</a></li> 
      {% endfor %}
    </ul>
  </div>
</nav>
        
{% for i in range(0,obj.index)%}<a name="{{i}}"><br><br><br><br><br><br><center><img src="Resources/{{i}}.svg" align="middle"></center>{% endfor %}<script src="jquery/jquery-2.0.3.min.js"></script>
</body>
</html>""")

In [227]:
with open('display.html', 'w') as file:
    file.write(TEMPLATE1.render(obj=obj))

# Breakout Visualizations

In [276]:
participants = G.nodes(data=True)

objs = []
for b in breakouts:
    learn = []
    teach = []
    
    for p in participants:
        if b in p[1]['learn']:
            learn.append(p[0].split(' ')[0])
            
        if b in p[1]['teach']:
            teach.append(p[0].split(' ')[0])
       
    
    learn = ', '.join(learn)
    teach = ', '.join(teach)
    o = {}
    o['learn'] = learn
    o['teach'] = teach
    o['name'] = b
    
    objs.append(o)

In [279]:
# Define jinja2 template
TEMPLATE2 = Template("""<html><head><head><link rel="stylesheet" href="http://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css">

<!-- jQuery library -->
<script src="https://ajax.googleapis.com/ajax/libs/jquery/1.12.4/jquery.min.js"></script>

<!-- Latest compiled JavaScript -->
<script src="http://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/js/bootstrap.min.js"></script></head>
<div class="container">
  <h2>Breakouts Table</h2>
  <table class="table">
    <thead>
      <tr>
        <th>Subjects</th>
        <th>Learners</th>
        <th>Teachers</th>
      </tr>
    </thead>
    <tbody>
    {% for i in objs%}
      <tr>
        <td>{{i.name}}</td>
        <td>{{i.learn}}</td>
        <td>{{i.teach}}</td>
      </tr>
    {% endfor %}
    </tbody>
  </table>
</div>
<script src="jquery/jquery-2.0.3.min.js"></script>
</body>
</html>""")

In [280]:
with open('breakouts.html', 'w') as file:
    file.write(TEMPLATE2.render(objs=objs))